<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/Spacing/Bert_Spacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 띄어쓰기 

출처 : 
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 01. 데이터 준비](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_01/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 02. 데이터 전처리](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_02/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 03. 모델](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_03/)
* [BERT를 이용한 한국어 띄어쓰기 모델 만들기 - 04. 학습](https://bhchoi.github.io/post/nlp/dev/bert_korean_spacing_04/)

## TPU를 사용한다면

 ( 이래도 안되는건 함정..? )

``` sql
!pip install cloud-tpu-client torch https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
```
``` python
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torchvision

# tpu
# Creates AlexNet for 10 classes
net = torchvision.models.alexnet(num_classes=10)

# Acquires the default Cloud TPU core and moves the model to it
device = xm.xla_device()
net = net.to(device)

```

# 패키지 설치 및 선언, GPU 사용 설정

In [ ]:
# !pip install mxnet
# !pip install gluonnlp tqdm
!pip install sentencepiece transformers torch
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'   # KoBERTTokenizer, 여기에선 tokenizer을 직접 선언했다.

In [ ]:
# !pip install tensorboard_logger
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import pytorch_lightning as pl
pl.__version__

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [7]:
from transformers import BertConfig, BertModel, AdamW
# import torch.optim as optim
# import gluonnlp as nlp

In [8]:
from tqdm import tqdm, tqdm_notebook
# from transformers.optimization import get_cosine_schedule_with_warmup
# from kobert.utils import get_tokenizer    # 아래에 직접 선언
# from kobert.pytorch_kobert import get_pytorch_kobert_model
from typing import Callable, List, Tuple
from seqeval.metrics import f1_score

In [4]:
import torch

# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

cuda:0


# 1. 학습 데이터 준비

100,000 lines 사용 ( headline, subheadline, content )

In [2]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Mounted at /content/drive


In [10]:
import os
content = []

for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              text = text.split('\n')
              for t in text :
                content.append(t)
        if len(content) > 100000 :
            break
                # if t[-1] != '.' :   # .으로 끝나지 않는 문장은 제목, 부제목으로 간주한다. ( 잘린 단어도 있을 수 있다. )
len(content)

100349

``` python
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
```


+ '.'으로 끝나지 않는 문장(headline, subheadline 포함) 100가지 살펴본 결과 모두 정상 문장이었다.
+ 하지만 PyKoSpacing로 띄어쓰기 검증을 했을 때 잘못된 띄어쓰기를 하였다.
+ PyKoSpacing는 ~다.로 끝나는 문장에는 띄어쓰기 결과가 거의 100% 정확도를 보이지만, 제목, 부제목과 같이 요약된 문장에는 좋지 못한 성능(60%)를 보인다.

In [5]:
# '.'으로 끝나지 않는 문장 100가지 살펴본 결과 모두 정상 문장이었다.

['원칙 있는 사회를 갈망한 노무현을 만나다', '수업 시간 도중 교수가 한 말이 발단이었다', '있는 그대로의 노무현을 알기 위해', '원칙이 승리하는 사회를 갈망한 노무현을 만나다', '이를 바로잡기 위해서는 어떡해야 할까?', '그러나 나는 이것이 객관적이고 보편적인 가치라고 생각하고 있습니다', '<노무현, 마지막 인터뷰 中에서>', '하지만 패배는 있지만 패배주의는 없습니다', '링컨을 존경했던 노무현 전 대통령에 대한 평가도 언젠가 호의적인 평가로 돌아설 수 있을까?', '아니면 영원히 부정적인 평가로 끝나고 말까?', '어머니가 만들어 준 맛있는 호떡이 생각나네요', '외로이 산장인데요', '주소가 없으니 경찰이 조서를 꾸미며 주소를 말하라하니 주소를 말할 수 있었겠나요?', '쌍용차 폭풍전야... 경찰-사측 음식반입도 금지', '100조 부자감세 12%만 줄여도 모든 학생 무상급식 가능', '3조 원만 있으면 초중고 모든 학생에게 무상급식 가능', '경기도 문제를 넘어 학교급식법 개정에 관심 가져야', '취학필수비무상설이 다수설이고 또 타당하다. (권영성, <헌법학원론>, 2002, 616쪽)', '헌법도 무상급식의 손을 들어준다', '하지만 수업료를 받을 수 없다', '과수원 주인, 1200여 학살영혼 구했다', '명석 용산리 2곳 발굴했지만 유골 나오지 않아', '아직도 정신 못차린 검찰', '국익이 먼저인가 사생활 보호가 먼저인가?', '정권의 검찰이 아닌 국민의 검찰이 돼야', '이명박 대통령도 말하지 않았는가?', '지금이야말로 국민의 검찰로 태어날 수 있는 적기다', '우리 옛말에 아픈 만큼 성숙한다고 하지 않았는가?', '태안살리기 무료시식회, 수산물 외지에서 조달 빈축', '뉴타운 청사진? 그럴 듯한데 돈이 문제지', '만안뉴타운 주민설명회 누구 위한 것이냐?', '질의응답부터 먼저', '해!', '시장 왔으니 우선 얘기부터 들어봐!', '진행이 안 되잖아!', '듣기 싫으면 네가 나가!', '모든 것을 열어놓고 여러분을 위한 사업

In [11]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, val = train_test_split(content[:100000], test_size=0.2, random_state=0, shuffle=True)
print(len(train), len(val))

80000 20000


# 2. 데이터 전처리

+ [KoBertTokenizer](#KoBertTokenizer)
+ [CorpusDataset](#CorpusDataset)
+ [Preprocessor](#Preprocessor)
+ [Config](#Config)

## KoBertTokenizer

[github](https://github.com/monologg/KoBERT-Transformers/blob/master/kobert_transformers/tokenization_kobert.py)

In [12]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"


class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [11]:
# test
# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
# tokenizer.encode("한국어 모델을 공유합니다.")

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

## CorpusDataset

In [19]:

# class CorpusDataset(Dataset):
#     # def __init__(self, data_path: str, config):
#     def __init__(self, sentences, config, tokenizer):
#         self.sentences = sentences
#         self.slot_labels = ["UNK", "PAD", "B", "I"]
#         # self._load_data(data_path)
#         self.config = config
#         self.tokenizer = tokenizer

#     def transform(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
#       """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

#       Args:
#           sentence: 문장
#           tags: 띄어쓰기 tagging

#       Returns:
#           feature를 리턴한다.
#           input_ids, attention_mask, token_type_ids, slot_labels
#       """

#       input_tokens = []
#       slot_label_ids = []
#       tags = self._get_tags(self.sentences)
#       # tokenize
#       for word, tag in zip(self.sentences, tags):
#           tokens = self.tokenizer(word)

#           if len(tokens) == 0:
#               tokens = self.tokenizer.unk_token

#           input_tokens.extend(tokens)

#           for i in range(len(tokens)):
#               if i == 0:
#                   slot_label_ids.extend([tag])
#               else:
#                   slot_label_ids.extend([self.tokenizer.pad_token_id])

#       # max_len보다 길이가 길면 뒤에 자르기
#       if len(input_tokens) > config.max_len - 2:
#           input_tokens = input_tokens[: config.max_len - 2]
#           slot_label_ids = slot_label_ids[: config.max_len - 2]

#       # cls, sep 추가
#       input_tokens = (
#           [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
#       )
#       slot_label_ids = [self.tokenizer.pad_token_id] + slot_label_ids + [self.tokenizer.pad_token_id]

#       # token을 id로 변환
#       input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

#       attention_mask = [1] * len(input_ids)
#       token_type_ids = [0] * len(input_ids)

#       # padding
#       pad_len = config.max_len - len(input_tokens)
#       input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
#       slot_label_ids = slot_label_ids + ([self.tokenizer.pad_token_id] * pad_len)
#       attention_mask = attention_mask + ([0] * pad_len)
#       token_type_ids = token_type_ids + ([0] * pad_len)

#       input_ids = torch.tensor(input_ids, dtype=torch.long)
#       attention_mask = torch.tensor(attention_mask, dtype=torch.long)
#       token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
#       slot_label_ids = torch.tensor(slot_label_ids, dtype=torch.long)

#       return input_ids, attention_mask, token_type_ids, slot_label_ids 
#     def _get_tags(self, sentence: List[str]) -> List[str]:
#         """문장에 대해 띄어쓰기 tagging을 한다.
#         character 단위로 분리하여 BI tagging을 한다.

#         Args:
#             sentence: 문장

#         Retrns:
#             문장의 각 토큰에 대해 tagging한 결과 리턴
#             ["B", "I"]
#         """

#         tags = []
#         for word in sentence:
#             for i in range(len(word)):
#                 if i == 0:
#                     tags.append("B")
#                 else:
#                     tags.append("I")
#         return tags

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = "".join(self.sentences[idx])
#         sentence = [s for s in sentence]
#         tags = self._get_tags(self.sentences[idx])
#         tags = [self.slot_labels.index(t) for t in tags]

#         (
#             input_ids,
#             attention_mask,
#             token_type_ids,
#             slot_label_ids, 
#         ) = self._transform(sentence, tags)

#         return input_ids, attention_mask, token_type_ids, slot_label_ids

In [13]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable[[List, List], Tuple]):
        self.sentences = []
        self.slot_labels = ["UNK", "PAD", "B", "I"]
        self.transform = transform
        self._load_data(sentences)

    def _load_data(self, sentences):
        """data를 file에서 불러온다.

        Args:
            data_path: file 경로
        """
        self.sentences = [sen.split() for sen in sentences]
        # with open(data_path, mode="r", encoding="utf-8") as f:
        #     lines = f.readlines()
        #     self.sentences = [line.split() for line in lines]

    def _get_tags(self, sentence: List[str]) -> List[str]:
        """문장에 대해 띄어쓰기 tagging을 한다.
        character 단위로 분리하여 BI tagging을 한다.

        Args:
            sentence: 문장

        Retrns:
            문장의 각 토큰에 대해 tagging한 결과 리턴
            ["B", "I"]
        """

        tags = []
        for word in sentence:
            for i in range(len(word)):
                if i == 0:
                    tags.append("B")
                else:
                    tags.append("I")
        return tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = "".join(self.sentences[idx])
        sentence = [s for s in sentence]
        tags = self._get_tags(self.sentences[idx])
        tags = [self.slot_labels.index(t) for t in tags]

        (
            input_ids,
            attention_mask,
            token_type_ids,
            slot_label_ids, 
        ) = self.transform(sentence, tags)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

## Preprocessor

In [14]:
from typing import List, Tuple

class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, sentence, tags
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging

        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """

        input_tokens = []
        slot_label_ids = []
					
        # tokenize
        for word, tag in zip(sentence, tags):
            tokens = self.tokenizer.tokenize(word)

            if len(tokens) == 0:
                tokens = self.tokenizer.unk_token

            input_tokens.extend(tokens)

            for i in range(len(tokens)):
                if i == 0:
                    slot_label_ids.extend([tag])
                else:
                    slot_label_ids.extend([self.pad_token_id])

        # max_len보다 길이가 길면 뒤에 자르기
        if len(input_tokens) > self.max_len - 2:
            input_tokens = input_tokens[: self.max_len - 2]
            slot_label_ids = slot_label_ids[: self.max_len - 2]

        # cls, sep 추가
        input_tokens = (
            [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
        )
        slot_label_ids = [self.pad_token_id] + slot_label_ids + [self.pad_token_id]

        # token을 id로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        # padding
        pad_len = self.max_len - len(input_tokens)
        input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
        slot_label_ids = slot_label_ids + ([self.pad_token_id] * pad_len)
        attention_mask = attention_mask + ([0] * pad_len)
        token_type_ids = token_type_ids + ([0] * pad_len)

        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        slot_label_ids = torch.tensor(slot_label_ids, dtype=torch.long)

        return input_ids, attention_mask, token_type_ids, slot_label_ids

## Config

파라미터 정의
yaml파일 대신에 class 사용

In [15]:
# yaml 파일 대신에 객체로 생성
class Config():
  def __init__(self) :
    self.task= 'korean_spacing_20210101'
    self.log_path= 'logs'
    self.bert_model =  'monologg/kobert'
    self.train_data_path= data_path+'/train.txt'
    self.val_data_path= data_path+'/val.txt'
    self.test_data_path= data_path+'/test.txt'
    self.max_len= 128
    self.max_len= 128
    self.train_batch_size= 64
    self.eval_batch_size= 64
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
# , tpu_cores=8
config = Config()

# 3. 모델

## SpacingBertModel

In [16]:
class SpacingBertModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = ["UNK", "PAD", "B", "I"]
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return self.linear(self.dropout(outputs[0]))

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels

        return {"loss": loss, "log": {"train_loss": loss}}

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        val_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)

        return {"val_loss": loss, "val_f1": val_f1}

    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_log = {
            "val_loss": val_loss,
            "val_f1": torch.stack([x["val_f1"] for x in outputs]).mean(),
        }

        return {"val_loss": val_loss, "progress_bar": tensorboard_log}

    def test_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            self(
              input_ids=input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids,
            ), slot_label_ids
        )

        test_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)

        return {"test_f1": test_f1, }

    def test_epoch_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()

        test_step_outputs = {
            "test_f1": test_f1,
        }

        return test_step_outputs

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def _calculate_loss(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)

        return loss

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

# 4. 학습

+ Preprocessor 선언, dataset 생성, 모델 선언
+ callbacks : TensorBoardLogger, ModelCheckpoint, EarlyStopping, LearningRateMonitor

## Preprocessor 선언, dataset 생성, 모델 선언

In [17]:
preprocessor = Preprocessor(config.max_len)

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [18]:
dataset = {}
dataset["train"] = CorpusDataset(train, preprocessor.get_input_features)
dataset["val"] = CorpusDataset(val, preprocessor.get_input_features)

# dataset["test"] = CorpusDataset(
#     config.test_data_path, preprocessor.get_input_features
# )

bert_finetuner = SpacingBertModel(config, dataset)

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [19]:
dataset

{'train': <__main__.CorpusDataset at 0x7f0563a9b610>,
 'val': <__main__.CorpusDataset at 0x7f0563a9b650>}

## callbacks

In [20]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=1, name=config.task
)


checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/"+ config.task, 
    filename="/{epoch}_{val_loss:35f}",
    verbose=True,
    monitor="val_loss",
    mode="min",
    save_top_k=3)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=3,
    verbose=False,
    mode="min",
)
lrmonitor_callback = LearningRateMonitor(logging_interval='step')

## Train

In [21]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[checkpoint_callback, early_stop_callback, lrmonitor_callback],
    logger=logger,
)

trainer.fit(bert_finetuner)
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type      | Params
--------------------------------------
0 | model   | BertModel | 92.2 M
1 | dropout | Dropout   | 0     
2 | linear  | Linear    | 3.1 K 
--------------------------------------
92.2 M    Trainable params
0         Non-trainable params
92.2 M    Total params
368.760   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: UNK seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasi

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Validating: 0it [00:00, ?it/s]

RuntimeError: ignored